In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn as nn


transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])  
])


train_dataset = datasets.ImageFolder(root="/Users/mertgenc/Desktop/aı_datasets/CNN/chest_xray/train", transform=transform)
val_dataset   = datasets.ImageFolder(root="/Users/mertgenc/Desktop/aı_datasets/CNN/chest_xray/val", transform=transform)
test_dataset  = datasets.ImageFolder(root="/Users/mertgenc/Desktop/aı_datasets/CNN/chest_xray/test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32)
test_loader  = DataLoader(test_dataset, batch_size=32)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

resnet = models.resnet18(pretrained=True)
num_features = resnet.fc.in_features
resnet.fc = nn.Linear(num_features, 2)

resnet = resnet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.0001)
epoch = 20

for epoch in range(epoch):
    resnet.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = resnet(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

resnet.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = resnet(inputs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {100 * correct / total:.2f}%")

/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/mertgenc/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████████████████████████████████| 44.7M/44.7M [00:09<00:00, 4.85MB/s]


In [14]:
import os

def jpg_dosya_sayisi(klasor_yolu):
    try:
        # Sadece .jpg uzantılı dosyaları filtrele
        jpg_dosyalar = [f for f in os.listdir(klasor_yolu) 
                        if os.path.isfile(os.path.join(klasor_yolu, f)) and f.lower().endswith('.jpeg')]
        
        print(f".jpg uzantılı dosya sayısı: {len(jpg_dosyalar)}")
    except FileNotFoundError:
        print("Belirttiğin klasör yolu bulunamadı.")
    except Exception as e:
        print(f"Bir hata oluştu: {e}")

# Örnek kullanım
klasor_yolu = r"/Users/mertgenc/Desktop/aı_datasets/CNN/chest_xray/train/PNEUMONIA"
jpg_dosya_sayisi(klasor_yolu)



.jpg uzantılı dosya sayısı: 3875


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2  
)

test_datagen = ImageDataGenerator(rescale=1./255)


train_dir = "/Users/mertgenc/Desktop/aı_datasets/CNN/chest_xray/train"
test_dir = "/Users/mertgenc/Desktop/aı_datasets/CNN/chest_xray/test"


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  
    batch_size=32,
    class_mode='binary', 
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Test generatorı
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)


base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))


for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)  
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  

model = Model(inputs=base_model.input, outputs=x)


model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

call_backs = [
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True),
    ModelCheckpoint('keras_model.h5', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, verbose=1)
]


history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=call_backs
)


test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.4f}")


#model.save("resnet50_chest_xray.h5")


Found 4173 images belonging to 2 classes.
Found 1043 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 883ms/step - accuracy: 0.7127 - loss: 0.6209

131/131 ━━━━━━━━━━━━━━━━━━━━ 149s 1s/step - accuracy: 0.7129 - loss: 0.6206 - val_accuracy: 0.7430 - val_loss: 0.5151 - learning_rate: 0.0010
Epoch 2/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 913ms/step - accuracy: 0.7526 - loss: 0.5132

131/131 ━━━━━━━━━━━━━━━━━━━━ 150s 1s/step - accuracy: 0.7526 - loss: 0.5131 - val_accuracy: 0.7613 - val_loss: 0.4606 - learning_rate: 0.0010
Epoch 3/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 976ms/step - accuracy: 0.7647 - loss: 0.4541

131/131 ━━━━━━━━━━━━━━━━━━━━ 164s 1s/step - accuracy: 0.7648 - loss: 0.4539 - val_accuracy: 0.7632 - val_loss: 0.4481 - learning_rate: 0.0010
Epoch 4/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7981 - loss: 0.4134

131/131 ━━━━━━━━━━━━━━━━━━━━ 184s 1s/step - accuracy: 0.7982 - loss: 0.4134 - val_accuracy: 0.7843 - val_loss: 0.4187 - learning_rate: 0.0010
Epoch 5/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8228 - loss: 0.3760

KeyboardInterrupt: 